#### This assignment may be worked individually or in pairs. Enter your name/names here:
    

In [1]:
#names here
Jacob Zillifro

SyntaxError: invalid syntax (<ipython-input-1-82b7c1838740>, line 2)

# Assignment 1: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1151` instances and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy (Accumulative label for the Messidor classes 1, 2, 3), 0 = no signs of Diabetic Retinopathy.


A few function prototypes are already given to you, please don't change those. You can add additional helper functions for your convenience. *Suggestion:* The dataset is substantially big, for the purpose of easy debugging work with a subset of the data and test your decision tree implementation on that.

#### Implementation: 
A few function prototypes are already given to you, please don't change those. You can add additional helper functions for your convenience. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 buckets. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5.
- For binary features [0/1] the threshold value will be 0.5. All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [2]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log2
from random import shuffle

In [3]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [4]:
def get_data(filename):
    data = []
    f = open(filename, "r")
    for l in f:
        attr = l.split(",")
        dp = DataPoint(int(attr[19]), [float(a) for a in attr[0:19]])
        data.append(dp)
    return data

In [5]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self):    # for debugging purposes
        if self.is_leaf:
            print ('Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree()
            self.right_child.printTree()

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [6]:
def make_prediction(tree_root, data_point):
#     your code goes here
    cur = tree_root
    while not cur.is_leaf:
        val = data_point.features[cur.feature_idx]
        if val < cur.thresh_val:
            cur = cur.left_child
        else:
            cur = cur.right_child
    return cur.prediction

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [7]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
#     your code goes here
    for dp in data:
        val = dp.features[feature_idx]
        if val < threshold:
            left_split.append(dp)
        else:
            right_split.append(dp)
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [8]:
def calc_entropy(data):
    entropy = 0.0
#     your code goes here
    g = 0
    l = 0
    total = len(data)
    if total == 0:
        return entropy
    for dp in data:
        if dp.label:
            g += 1
        else:
            l += 1
    if g == 0 or l == 0:
        return entropy
    g_prob = (g/total)
    l_prob = (l/total)
    entropy = ( ((g_prob)*log2(g_prob)*-1) + ((l_prob)*log2(l_prob)*-1) )
    return entropy

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [9]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = None
#     your code goes here
    test_vals = set()
    sdata = sorted(data, key=lambda dp: dp.features[feature_idx])
    test_vals.add(sdata[0].features[feature_idx])
    test_vals.add(sdata[-1].features[feature_idx])
    i = 1
    while i < len(sdata):
        if sdata[i].label != sdata[i-1].label or sdata[i].features[feature_idx] != sdata[i-1].features[feature_idx]:
            mid = (sdata[i].features[feature_idx] + sdata[i-1].features[feature_idx])/2
            test_vals.add(mid)
        i+=1
    for t in test_vals:
        split = split_dataset(data, feature_idx, t)
        g = calc_entropy(data) - calc_entropy(split[0]) - calc_entropy(split[1])
        if g >= best_info_gain:
            best_info_gain = g
            best_thresh = t
    return (best_info_gain, best_thresh)

Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [10]:
def identify_best_split(data):
    if len(data) < 2:
        return (None, None)
    best_feature = None
    best_thresh = None
#     your code goes here
    best_gain = 0.0
    for i in range(len(data[0].features)):
        b_t = calc_best_threshold(data, i)
        g = b_t[0]
        t = b_t[1]
        if g > best_gain:
            best_feature = i
            best_thresh = t
            best_gain = g
    return (best_feature, best_thresh)

Q7. Implement the function `createLeafNode` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [11]:
def createLeafNode(data):
#     your code goes here
    g = 0
    l = 0
    for dp in data:
        if dp.label == 1:
            g+=1
        else:
            l+=1
            
    result = TreeNode()
    result.is_leaf = True
    if g > l:
        result.prediction = 1
    else:
        result.prediction = 0
            
            
    return result

Q8. Implement the `createDecisionTree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [12]:
def createDecisionTree(data, max_levels):
#     your code goes here
    node = None
    if max_levels > 1:
        best_split = identify_best_split(data)
        if best_split[0] is None:
            node = createLeafNode(data)
        else:
            node = TreeNode()
            node.is_leaf = False
            node.feature_idx = best_split[0]
            node.thresh_val = best_split[1]
            data_split = split_dataset(data, best_split[0], best_split[1])
            node.left_child = createDecisionTree(data_split[0], max_levels-1)
            node.right_child = createDecisionTree(data_split[1], max_levels-1)
    else:
        node = createLeafNode(data)
    return node

Q9. Given a test set, the function `calcAccuracy` returns the accuracy of the classifier. You'll use the `makePrediction` function for this.

In [13]:
def calcAccuracy(tree_root, data):
#     your code goes here
    sum = 0
    for dp in data:
        p = make_prediction(tree_root, dp)
        if p == dp.label:
            sum += 1
    return (sum/len(data))

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the recall and precision of the model. Also display the confusion matrix.

In [14]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")
l = len(d)
t_l = l//5

acc_sum = 0
for i in range(5):
    # partition data into train_set and test_set
    train_set = d[:i*t_l] + d[i*t_l+t_l:]
    test_set = d[i*t_l:i*t_l+t_l]

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # create the decision tree
    start = time.time()
    tree = createDecisionTree(train_set, 10)
    end = time.time()
    print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calcAccuracy(tree, test_set)
    acc_sum += accuracy
    print ('The accuracy on the test set ' + str(i) + ' is ', str(accuracy * 100.0))
#     tree.printTree()
acc_avg = acc_sum/5
print ('The average accuracy on the test sets is ', str(acc_avg * 100.0))

Training set size: 921
Test set size    : 230
Time taken: 16.87299370765686
The accuracy on the test set 0 is  56.086956521739125
Training set size: 921
Test set size    : 230
Time taken: 19.10603976249695
The accuracy on the test set 1 is  53.04347826086957
Training set size: 921
Test set size    : 230
Time taken: 16.58595848083496
The accuracy on the test set 2 is  52.17391304347826
Training set size: 921
Test set size    : 230
Time taken: 14.415998935699463
The accuracy on the test set 3 is  54.78260869565217
Training set size: 921
Test set size    : 230
Time taken: 11.723037958145142
The accuracy on the test set 4 is  50.0
The average accuracy on the test sets is  53.21739130434783
